## The version below works, but it keeps it in greyscale and modifies the image quite a bit

In [51]:
import cv2
import numpy as np
import base64

def embed_base64_string(img, base64_string):
    # Convert the Base64 string to binary
    binary_string = base64.b64decode(base64_string)
    binary_array = np.frombuffer(binary_string, dtype=np.uint8)

    # Pad the binary data to match the length of the image
    num_pixels = img.shape[0] * img.shape[1]
    num_bits = len(binary_array) * 8
    if num_bits > num_pixels:
        raise ValueError("Message too long to embed in image")
    num_zeros = num_pixels - num_bits
    padded_data = np.pad(binary_array, (0, num_zeros), mode='constant')

    # Perform domain transform on the image
    #img_dct = cv2.dct(img.astype(np.float32))
    #print(img_dct)
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_float = np.float32(img_gray) / 255.0
    img_dct = cv2.dct(img_float)
    img_dct_flat = img_dct.flatten()

    # Embed the data into the DCT coefficients
    img_dct_embed = img_dct_flat.copy()
    for i in range(len(padded_data)):
        if padded_data[i]:
            img_dct_embed[i] = img_dct_flat[i] + 1
        else:
            img_dct_embed[i] = img_dct_flat[i] - 1

    # Reshape the DCT coefficients and perform inverse DCT
    img_dct_embed_reshape = np.reshape(img_dct_embed, img_dct.shape)
    img_idct = cv2.idct(img_dct_embed_reshape)

    # Convert the image back to BGR format and return
    img_embed = np.uint8(img_idct * 255.0)
    #result = cv2.cvtColor(img_embed, cv2.COLOR_GRAY2BGR)
    return img_embed

# Example usage:
# Load the input image
img = cv2.imread("picture.jpg")

# Define the Base64 string to be embedded
data_string = 'Hello World!'
base64_string = base64.b64encode(data_string.encode())

# Embed the string in the image
result = embed_base64_string(img, base64_string)

# Save the output image
cv2.imwrite("output_image.jpg", result)

True

## Everything below still doesn't work

In [48]:
import hashlib

def hash_video(filename):
    # Initialize the SHA-256 hash object
    hash_obj = hashlib.sha256()

    # Read the contents of the file in chunks
    with open(filename, 'rb') as f:
        while True:
            chunk = f.read(4096)  # Read 4 KB at a time
            if not chunk:
                break
            hash_obj.update(chunk)

    # Get the hexadecimal digest of the hash
    hash_value = hash_obj.hexdigest()

    return hash_value

digest = hash_video("video.mp4")
print(digest)
len(digest)

66850ecc12191c682ed74e133b1a4eba65aff358107753d5debcc90bf46e5cb6


64

In [42]:
def recover_base64_string(img):
    # Perform domain transform on the image
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_float = np.float32(img_gray) / 255.0
    img_dct = cv2.dct(img_float)
    img_dct_flat = img_dct.flatten()

    # Recover the data from the DCT coefficients
    recovered_data = []
    for i in range(len(img_dct_flat)):
        if img_dct_flat[i] % 2 == 0:
            recovered_data.append(0)
        else:
            recovered_data.append(1)
    
    # Convert the recovered data to binary and then to a Base64 string
    recovered_binary = np.packbits(recovered_data)
    recovered_string = base64.b64encode(recovered_binary).decode('utf-8')
    
    return recovered_string

img = cv2.imread("output_image.jpg")
recover_base64_string(img)

'...'

In [29]:
import cv2
import numpy as np
import base64

def embed_message_in_video(video_path, message):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the video frame dimensions
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Compute the maximum message size that can be embedded in the video
    max_message_size = (frame_width * frame_height) // 2

    # Check if the message can be embedded in the video
    message_bytes = message.encode('ascii')
    message_b64 = base64.b64encode(message_bytes)
    message_size = len(message_b64)
    if message_size > max_message_size:
        raise ValueError('Message too large to embed in video')

    # Precompute the message bitstream and the mask for embedding the message in the blue channel
    message_bits = np.unpackbits(np.frombuffer(message_b64, dtype=np.uint8))
    message_bits_padded = np.pad(message_bits, (0, max_message_size - message_size))
    blue_channel_mask = np.tile(np.array([254, 1], dtype=np.uint8), (frame_height, frame_width // 2))

    # Loop over the frames of the video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(video_path[:-4] + '_embedded' + video_path[-4:], fourcc, cap.get(cv2.CAP_PROP_FPS),
                             (frame_width, frame_height), True)
    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # Check if the frame was successfully read
        if not ret:
            break

        # Embed the message in the frame using the domain transform method
        # Here, we use a simple example where we embed the message in the blue channel
        blue_channel = frame[:, :, 0]
        blue_channel_embedded = np.bitwise_or(np.bitwise_and(blue_channel, blue_channel_mask),
                                              np.left_shift(message_bits_padded.reshape((-1, frame_width // 2)), 1))

        # Copy the embedded blue channel back to the frame
        frame[:, :, 0] = blue_channel_embedded

        # Write the frame to the new video file
        writer.write(frame)

    # Release the video capture object and close the video writer
    cap.release()
    writer.release()

    print(f'Message embedded in video: {message}')

Error: Incorrect padding

In [64]:
img1 = cv2.imread("picture.jpg")
img_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img_float = np.float32(img_gray) / 255.0
img_dct = cv2.dct(img_float)
cv2.imwrite("dct.jpg", img_dct)

True

In [57]:
img_gray.shape

(903, 906)

In [61]:
img_dct.shape

(903, 906)